<a href="https://colab.research.google.com/github/henrykohl/Machine-Learning-demo-repo/blob/master/jovian/cross-entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<span style="font-size:14px; font-family:Arial;">Cross-Entropy 在資料分類上，有著非常重要的功能。網路上關於在Pytorch中，如何使用封裝好的函數，獲得Cross Entropy的文章相當多，有的過於簡單，有的又過於複雜，深入淺出難易適中，還能從定義聯繫到實例示範（例如，手動計算Cross Entropy與利用Pytorch所獲得的結果，並比較兩結果）的文章更是不多。本文將從Cross Entropy的定義開始，列出出常看到的定義（舉兩個看起來不同的來說明），接著將用一個實際例子，先依據理論手動計算Cross Entropy，然後使用Pytorch，利用不同的封裝函數，示範如何求得Cross Entropy，再互相比較驗證此兩結果。</span>

<span style="font-size:14px; font-family:Arial;">定義一：$D(\hat Y, Y)$與定義二：$D(S, L)$ ，兩者的差異，在於前者是mean，而後者是sum，$N$是輸入資料的筆數(也就是batch size)，$\hat Y_i$是指第$i$筆資料的預測機率分布向量（i.e., $i$是此筆資料的index），例如第$i$筆資料的預測機率分布$\hat Y_i$是$[0.7, 0.2, 0.1]$，此向量的維數是3，表示number of classes：$C$=3（e.g., 0表示第一類，1表示第二類，2表示第三類），$Y_i$是第$i$筆資料的實際類別標籤（e.g.,若$Y_i$的實際類別為"0"，那就是第一類），$Y_i$的one-hot encoded label被表示成：$[1,0,0]$。  
再看定義二，$S$就是sample（有另一含意softmax之後會談到）, $S_i$是第$i$筆資料的預測機率分布向量（如同$\hat Y_i$），$L$就是label，$L_i$是第$i$筆資料的實際類別標籤（如同$Y_i$）。注意，定義二中的 <font color="red">$y$</font>，$S(y)$是假設只有一筆資料，如果有多筆資料，應該要表示成$S(y_i)$，$S(y_i)$才等於$Y_i$，之後用實例說明會更為清楚，$y_i$是第$i$筆資料的類模型權重向量（向量中的值可能有負值，可能大於零，也可能小於零）。</span>

<span style="font-size:14px; font-family:Arial;">在分類問題中，通過訓練logistic regression model的過程，對每一個training data，我們會得到此data的類權重分布向量，也就是 <font color="red">$y$</font>，現在舉一實例，假設現在的training data set(dw)，包含五筆的權重分布向量，$N$=5，number of classes是3，$C$=3，現在用pytorcht隨機產生。</span>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0)
dw = torch.randn(5, 3)
dw

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820],
        [-0.8567,  1.1006, -1.0712]])

In [3]:
dw.shape

torch.Size([5, 3])

dw.shape，就是（$N$, $C$），有5筆權重向量或是Scores/Logits向量，類別數為3，第一筆 <font color="red">$y$</font> 就是dw[0]，第二筆 <font color="red">$y$</font> 就是dw[1]，...。在權重向量中，有正值有負值，可能有大於1，也可能小於1。因此權重向量需要透過softmax處理，轉成機率分布向量。softmax的定義如下：

<br>

$S(y_i)=\Large \frac{e^{y_{ij}}}{\sum_{j}^{C} e^{y_{ij}}}$

<br>

$y_{ij}$ 是指第$i$筆權重向量中的第$j$個的值。根據softmax的定義，可以算出$S(y_i)$，$0\leq i< N=5$

<br>

$S(y_0)=\Large [\frac{e^{dw[0,0]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}, \frac{e^{dw[0,1]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}, \frac{e^{dw[0,2]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}]$


$S(y_1)=\Large [\frac{e^{dw[1,0]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}, \frac{e^{dw[1,1]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}, \frac{e^{dw[1,2]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}]$

$S(y_2)=\Large [\frac{e^{dw[2,0]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}, \frac{e^{dw[2,1]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}, \frac{e^{dw[2,2]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}]$

$S(y_3)=\Large [\frac{e^{dw[3,0]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}, \frac{e^{dw[3,1]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}, \frac{e^{dw[3,2]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}]$

$S(y_4)=\Large [\frac{e^{dw[4,0]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}, \frac{e^{dw[4,1]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}, \frac{e^{dw[4,2]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}]$

<br>

在Pytorch中，可以用torch.nn.functional.softmax或是torch.nn.Softmax（注意大小寫）兩種方式。

In [4]:
# torch.nn.functional.softmax
fsoft = F.softmax(dw, dim=1)
fsoft

tensor([[0.8446, 0.1349, 0.0205],
        [0.7511, 0.1438, 0.1051],
        [0.3484, 0.5382, 0.1134],
        [0.2762, 0.2277, 0.4961],
        [0.1125, 0.7967, 0.0908]])

In [5]:
# torch.nn.Softmax
nsoft = nn.Softmax(dim=1)
nsoft(dw)

tensor([[0.8446, 0.1349, 0.0205],
        [0.7511, 0.1438, 0.1051],
        [0.3484, 0.5382, 0.1134],
        [0.2762, 0.2277, 0.4961],
        [0.1125, 0.7967, 0.0908]])

fsoft與nsoft的結果完全一致。接著是很簡單的一步驟，對$S(y_i)$取log，注意在Pytorch中log是以$e$為基底（log是ln）：

<br>

$log(S(y_0))$=$\Large [{\small log}\frac{e^{dw[0,0]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}, {\small log}\frac{e^{dw[0,1]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}, {\small log}\frac{e^{dw[0,2]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}]$

<br>

$log(S(y_1))$=$\Large [{\small log}\frac{e^{dw[1,0]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}, {\small log}\frac{e^{dw[1,1]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}, {\small log}\frac{e^{dw[1,2]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}]$

<br>

$log(S(y_2))$=$\Large [{\small log}\frac{e^{dw[2,0]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}, {\small log}\frac{e^{dw[2,1]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}, {\small log}\frac{e^{dw[2,2]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}]$

<br>

$log(S(y_3))$=$\Large [{\small log}\frac{e^{dw[3,0]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}, {\small log}\frac{e^{dw[3,1]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}, {\small log}\frac{e^{dw[3,2]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}]$

<br>

$log(S(y_4))$=$\Large [{\small log}\frac{e^{dw[4,0]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}, {\small log}\frac{e^{dw[4,1]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}, {\small log}\frac{e^{dw[4,2]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}]$

In [6]:
logfsoft = torch.log(fsoft)
logfsoft

tensor([[-0.1689, -2.0033, -3.8886],
        [-0.2862, -1.9392, -2.2532],
        [-1.0543, -0.6196, -2.1769],
        [-1.2865, -1.4797, -0.7011],
        [-2.1846, -0.2273, -2.3991]])

In [7]:
lognsoft = torch.log(nsoft(dw))
lognsoft

tensor([[-0.1689, -2.0033, -3.8886],
        [-0.2862, -1.9392, -2.2532],
        [-1.0543, -0.6196, -2.1769],
        [-1.2865, -1.4797, -0.7011],
        [-2.1846, -0.2273, -2.3991]])

上述兩個步驟在Pytorch中可以合併成一個步驟

In [8]:
# torch.nn.functional
log_fsoft = F.log_softmax(dw, dim=1)
log_fsoft

tensor([[-0.1689, -2.0033, -3.8886],
        [-0.2862, -1.9392, -2.2532],
        [-1.0543, -0.6196, -2.1769],
        [-1.2865, -1.4797, -0.7011],
        [-2.1846, -0.2273, -2.3991]])

In [9]:
# torch.nn
log_nsoft = nn.LogSoftmax(dim=1)
log_nsoft(dw)

tensor([[-0.1689, -2.0033, -3.8886],
        [-0.2862, -1.9392, -2.2532],
        [-1.0543, -0.6196, -2.1769],
        [-1.2865, -1.4797, -0.7011],
        [-2.1846, -0.2273, -2.3991]])

已知 資料標籤為 **labels**: `[2,2,1,0,1]`，下一個步驟是把 **`labels`** 與 上面取$log$ 的結果 (`log_fsoft` 或 `log_nsoft(dw)`) ，執行 Element-wise multiplication 然後再做 Summation.

In [10]:
labels = torch.tensor([2,2,1,0,1])
labels

tensor([2, 2, 1, 0, 1])

假設 $L_0$ 的 label是 **2**, $L_1$ 的 label是 **2**, $L_2$ 的 label是 **1**, $L_3$ 的 label是 **0**, $L_4$ 的 label是 **1**，則他們的one-hot encoded 的結果：

<br>

$$L_0=[0,0,1]$$ <br>
$$L_1=[0,0,1]$$ <br>
$$L_2=[0,1,0]$$ <br>
$$L_3=[1,0,0]$$ <br>
$$L_4=[0,1,0]$$
        
        
        

In [11]:
onehotcode = F.one_hot(labels)
onehotcode

tensor([[0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0]])

In [12]:
ls = onehotcode * log_fsoft
ls

tensor([[-0.0000, -0.0000, -3.8886],
        [-0.0000, -0.0000, -2.2532],
        [-0.0000, -0.6196, -0.0000],
        [-1.2865, -0.0000, -0.0000],
        [-0.0000, -0.2273, -0.0000]])

In [13]:
ls = onehotcode * log_nsoft(dw)
ls

tensor([[-0.0000, -0.0000, -3.8886],
        [-0.0000, -0.0000, -2.2532],
        [-0.0000, -0.6196, -0.0000],
        [-1.2865, -0.0000, -0.0000],
        [-0.0000, -0.2273, -0.0000]])

最後一步，算出Cross-Entropy的結果（以mean的型態）：

<br>

$$Cross-Entropy=-\frac{1}{5}(L_0log(S(y0))+L_1log(S(y1))+L_2log(S(y2))+L_3log(S(y3))+L_4log(S(y4)))$$

<br>

$$=-\small\frac{1}{5}[0,0,1]\cdot\Large [{\small log}\frac{e^{dw[0,0]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}},{\small log}\frac{e^{dw[0,1]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}},{\small log}\frac{e^{dw[0,2]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}]$$

$$-\small\frac{1}{5}[0,0,1]\cdot\Large [{\small log}\frac{e^{dw[1,0]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}},{\small log}\frac{e^{dw[1,1]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}},{\small log}\frac{e^{dw[1,2]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}]$$

$$-\small\frac{1}{5}[0,1,0]\cdot\Large [{\small log}\frac{e^{dw[2,0]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}},{\small log}\frac{e^{dw[2,1]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}},{\small log}\frac{e^{dw[2,2]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}]$$

$$-\small\frac{1}{5}[1,0,0]\cdot\Large [{\small log}\frac{e^{dw[3,0]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}},{\small log}\frac{e^{dw[3,1]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}},{\small log}\frac{e^{dw[3,2]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}]$$

$$-\small\frac{1}{5}[0,1,0]\cdot\Large [{\small log}\frac{e^{dw[4,0]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}},{\small log}\frac{e^{dw[4,1]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}},{\small log}\frac{e^{dw[4,2]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}]$$

<br>

$$=-\small\frac{1}{5}{\small log}\Large\frac{e^{dw[0,2]}}{e^{dw[0,0]}{\small +}e^{dw[0,1]}{\small +}e^{dw[0,2]}}-\small\frac{1}{5}{\small log}\Large\frac{e^{dw[1,2]}}{e^{dw[1,0]}{\small +}e^{dw[1,1]}{\small +}e^{dw[1,2]}}-\small\frac{1}{5}{\small log}\Large\frac{e^{dw[2,1]}}{e^{dw[2,0]}{\small +}e^{dw[2,1]}{\small +}e^{dw[2,2]}}$$
$$-\small\frac{1}{5}{\small log}\Large\frac{e^{dw[3,0]}}{e^{dw[3,0]}{\small +}e^{dw[3,1]}{\small +}e^{dw[3,2]}}-\small\frac{1}{5}{\small log}\Large\frac{e^{dw[4,1]}}{e^{dw[4,0]}{\small +}e^{dw[4,1]}{\small +}e^{dw[4,2]}}$$

<br>

$$=1.6550$$

In [14]:
sumls = torch.sum(ls, dim=1)
ce = -torch.mean(sumls)
sumls, ce

(tensor([-3.8886, -2.2532, -0.6196, -1.2865, -0.2273]), tensor(1.6550))

<font color="red">注意 `torch.mean` 是對應 $\frac{1}{N} \sum_{i}$，而 `torch.sum( ,dim=1)`是對應同一筆資料（同一個row）的 label one hot code 與 log 結果 的 inner product</font>

上述步驟，在Pytorch中可以使用torch.nn.functional.nll_loss或是torch.nn.NLLLoss，注意在Pytroch裡，實際標籤$L_i$或是$Y_i$是不需要使用one-hot encoded label，所以$L$可以直接表示成一個向量[2,2,1,0,1]，向量的index就是資料的編號$i$，向量中的值就是`類`的標籤。

In [29]:
# torch.nn.functional.nll_loss
nll_out = F.nll_loss(log_fsoft, labels)
nll_out

tensor(1.6550)

In [30]:
# torch.nn.functional.nll_loss
NLLLoss_out = torch.nn.NLLLoss()
NLLLoss_out(log_nsoft(dw),labels)

tensor(1.6550)

實際上，上述步驟在Pytorch中，有了權重向量（dw）與已知的分類標籤（labels），只需要一個步驟，使用`torch.nn.functional.cross_entropy`或是`torch.nn.CrossEntropyLoss`，就可求出Cross-Entroy的值

In [ ]:
# torch.nn.functional.cross_entropy
cross_entropy = F.cross_entropy(dw, labels)
cross_entropy

tensor(0.5958)

In [ ]:
# torch.nn.CrossEntropyLoss
CrossEntropyLoss = torch.nn.CrossEntropyLoss()
CrossEntropyLoss(dw,labels)

tensor(0.5958)

## 注意
在torch.nn.functional.cross_entropy或是torch.nn.CrossEntropyLoss沒有加上reduction參數，則預設reduction='mean'

In [45]:
# torch.nn.functional.cross_entropy
cross_entropy = F.cross_entropy(dw, labels, reduction='mean')
cross_entropy

tensor(1.6550)

In [46]:
# torch.nn.CrossEntropyLoss
CrossEntropyLoss = torch.nn.CrossEntropyLoss(reduction='mean')
CrossEntropyLoss(dw, labels)

tensor(1.6550)

reduction='sum'表示求總和，reduction='none'表示呈現每一筆traning data的cross entropy

In [47]:
# torch.nn.functional.cross_entropy
cross_entropy = F.cross_entropy(dw, labels, reduction='none')
cross_entropy

tensor([3.8886, 2.2532, 0.6196, 1.2865, 0.2273])

In [48]:
# torch.nn.CrossEntropyLoss
CrossEntropyLoss = torch.nn.CrossEntropyLoss(reduction='none')
CrossEntropyLoss(dw, labels)

tensor([3.8886, 2.2532, 0.6196, 1.2865, 0.2273])

此圖(圖三)乍看之下，當$N=1$時，$\hat Y=[0.1, 0.5, 0.4]$且$Y=[0,1,0]$，似乎很好理解，但當$N>1$，容易有誤解，$N$是batch size，不是number of class，所以用$N>1$的範例來解釋圖三，$\hat Y$與$Y$不是只有一個向量，而是$N\times C$的矩陣，所以圖三中$[0.1, 0.5, 0.4]$是$\hat Y$中某一個row向量，而$Y$原本是$N\times 1$的類標籤(lable encoding)的一維向量，用one-hot encoding將一維向量，轉成$N\times C$的2維陣列，圖三中$[0, 1, 0]$其實是$Y$中某一個row向量，圖三的$j$是batch的index而不是當成class的index，為了更清楚說明，用$i$當成batch的index，$j$換成class的index。圖三的equation可以更清楚地表示成：
$$D(\hat Y, Y)=-\sum_{i=0}^{N-1}\sum_{j=0}^{C-1} y_{ij}ln(\hat y_{ij})$$

# Dimensions greater than **2** (補充)

* 前一章節，**Cross Entropy**實驗用的data維度是**2**，`F.softmax`或`nn.Softmax`是基於`dim=1`。
* 本節示範當實驗用的data維度是**3**時，執行`F.softmax`或`nn.Softmax`還是（同樣）基於`dim=1`，(而非`dim=2`)。

In [3]:
"""
若(case 1)
Letters=3
Samples=2
C=4
而非(case 2)
N=3
C=2
d1=4
"""
torch.manual_seed(0)
dw = torch.randn(3, 2, 4)
dw

tensor([[[-1.1258, -1.1524, -0.2506, -0.4339],
         [ 0.8487,  0.6920, -0.3160, -2.1152]],

        [[ 0.4681, -0.1577,  1.4437,  0.2660],
         [ 0.1665,  0.8744, -0.1435, -0.1116]],

        [[ 0.9318,  1.2590,  2.0050,  0.0537],
         [ 0.6181, -0.4128, -0.8411, -2.3160]]])

In [4]:
labels = torch.tensor([[[0.,1.,0.,0.],[0.,0.,1.,0.]],
             [[1.,0.,0.,0.],[0.,0.,0.,1.]],
             [[0.,0.,1.,0.],[0.,1.,0.,0.]]])
labels

tensor([[[0., 1., 0., 0.],
         [0., 0., 1., 0.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 1.]],

        [[0., 0., 1., 0.],
         [0., 1., 0., 0.]]])

In [5]:
F.cross_entropy(dw, labels), nn.CrossEntropyLoss()(dw, labels)

(tensor(0.5059), tensor(0.5059))

## CASE 1

**不**能直接使用`F.cross_entropy`或`nn.CrossEntropyLoss`

In [14]:
ce_sum0 = []
ce_sum1 = []
for letter, label in zip(dw, labels):
  fs = F.softmax(letter, dim=1)
  ylogyh = label*torch.log(fs)
  print(ylogyh)
  sumlog0 = torch.sum(ylogyh, dim=0)
  sumlog1 = torch.sum(ylogyh, dim=1)
  ce_sum0.append(sumlog0)
  ce_sum1.append(sumlog1)
torch.stack(ce_sum0), -torch.mean(torch.stack(ce_sum0))

tensor([[-0.0000, -1.8783, -0.0000, -0.0000],
        [-0.0000, -0.0000, -1.9616, -0.0000]])
tensor([[-1.6103, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -1.7867]])
tensor([[-0.0000, -0.0000, -0.6721, -0.0000],
        [-0.0000, -1.5270, -0.0000, -0.0000]])


(tensor([[ 0.0000, -1.8783, -1.9616,  0.0000],
         [-1.6103,  0.0000,  0.0000, -1.7867],
         [ 0.0000, -1.5270, -0.6721,  0.0000]]),
 tensor(0.7863))

In [15]:
torch.stack(ce_sum1), -torch.mean(torch.stack(ce_sum1))

(tensor([[-1.8783, -1.9616],
         [-1.6103, -1.7867],
         [-0.6721, -1.5270]]),
 tensor(1.5727))

## CASE 2

能直接使用`F.cross_entropy`或`nn.CrossEntropyLoss`

In [6]:
ce = F.cross_entropy(dw, labels)
cen = F.cross_entropy(dw, labels, reduction='none')
cen, ce

(tensor([[-0.0000, 1.9912, 0.7264, -0.0000],
         [0.5537, -0.0000, -0.0000, 0.8997],
         [-0.0000, 1.8440, 0.0564, -0.0000]]),
 tensor(0.5059))

手動計算 Cross Entropy

In [7]:
fsoft = F.softmax(dw, dim=1)
fsoft

tensor([[[0.1219, 0.1365, 0.5164, 0.8431],
         [0.8781, 0.8635, 0.4836, 0.1569]],

        [[0.5748, 0.2627, 0.8302, 0.5933],
         [0.4252, 0.7373, 0.1698, 0.4067]],

        [[0.5778, 0.8418, 0.9451, 0.9145],
         [0.4222, 0.1582, 0.0549, 0.0855]]])

In [9]:
ylogyh = labels * torch.log(fsoft)
sumlog = torch.sum(ylogyh, dim=1)
cem = -torch.mean(sumlog)
sumlog, cem

(tensor([[ 0.0000, -1.9912, -0.7264,  0.0000],
         [-0.5537,  0.0000,  0.0000, -0.8997],
         [ 0.0000, -1.8440, -0.0564,  0.0000]]),
 tensor(0.5059))